# Numeric Integration 2

### Uncertainty on The trapezoid rule

Recall the integral we've been using for examples, $\int_{0}^{2} ( x^4 - 2x + 1)dx$. Eq. 5.28 suggests we can estimate an uncertainty on our integral by integrating the function twice, one with $N$ steps and once with $2N$. Using $N=10$ and $N=20$ estimate the accuracy of your integral. How does it compare to the actual discrepancy with the known true value?

In [3]:
# code
import numpy as np

def f_1(x):
    return x**4 - 2*x + 1

def trapezoidSum(function, a, b, N_steps):
    h = (b-a) / N_steps
    total = 0.5*h * (function(a) + function(b))

    for k in range(1,N_steps):
        total += h * function(a + k*h)

    return total

print(trapezoidSum(f_1, 0, 2, 10))
print(trapezoidSum(f_1, 0, 2, 20))

print(np.abs((trapezoidSum(f_1, 0, 2, 10) - trapezoidSum(f_1, 0, 2, 20)) / 3))

4.506560000000001
4.426660000000002
0.026633333333333137


### Adaptive Trapezoid Rule

Sec 5.3 outlines a method for iteratively doubling the number of steps in a trapezoid rule until a desired precision is achieved. Write a function to implement this method for our test integral, $\int_{0}^{2} ( x^4 - 2x + 1)dx$, until a desired precision is reached. Choose your own goal.

In [11]:
# code
def AdaptiveTrapSum(function, a, b, N_steps = 10, tolerance = 0.0001, recursing = True, step_skip = 1, last_total = 0):
    h = (b-a) / N_steps
    if last_total == 0:
        total = 0.5*h * (function(a) + function(b))
    else:
        total = last_total

    for k in range(1, N_steps, step_skip):
        total += h * function(a + k*h)

    if recursing == True:
        trap_error = np.abs(total - AdaptiveTrapSum(function, a, b, N_steps=2*N_steps, recursing=False)) / 3
        print(f"{N_steps} steps, error: {trap_error}")
        
        if trap_error >= tolerance:
            total = AdaptiveTrapSum(function, a, b, N_steps=2*N_steps, tolerance=tolerance, recursing=True, step_skip=2, last_total=total/2)

    return total

print(AdaptiveTrapSum(f_1, 0, 2))

10 steps, error: 0.026633333333333137
20 steps, error: 0.006664583333333122
40 steps, error: 0.0016665364583333304
80 steps, error: 0.00041665852864565994
160 steps, error: 0.0001041661580408378
320 steps, error: 2.604163487873734e-05
4.400104166564945


With your method established in principle, use the same function or write a new one to evaluate the integral $\int_{0}^{1} \sin^2 \sqrt{100x} dx$ to a precision of $\epsilon \sim 10^{-6}$. Begin with a single slice and work your way up to two, four, eight, etc. At each step, print the number of slices and the error.

The correct answer is around 0.45.

In [7]:
# code
def f_2(x):
    return np.sin(np.sqrt(100 * x))**2

print(AdaptiveTrapSum(f_2, 0, 1, N_steps=1, tolerance=1e-6))

1 steps, error: 0.05908414108660753
2 steps, error: 0.06235031430561896
4 steps, error: 0.036428467415027734
8 steps, error: 0.009035306938832885
16 steps, error: 0.00610376549757428
32 steps, error: 0.0018327551426353301
64 steps, error: 0.000478524385808754
128 steps, error: 0.00012092069347964991
256 steps, error: 3.0311066141097687e-05
512 steps, error: 7.582826918632139e-06
1024 steps, error: 1.8960230754871965e-06
2048 steps, error: 4.7402554131936725e-07
0.4558306362016466


Repeat the previous exercise using the adaptive Simpson's Rule. You should find signficantly fewer steps are needed.

In [14]:
# code

def AdaptiveSimpSum(function, a, b, N_steps = 10, tolerance = 0.0001, recursing = True, last_total = 0):
    h = (b-a) / N_steps
    total_T, total_I = 0, 0
    if last_total == 0:
        total_S = (1/3) * (function(a) + function(b))
        for k in range(2, N_steps-1, 2):
            total_S += (2/3) * function(a + k*h)
    else:
        total_S = last_total

    for k in range(1, N_steps, 2):
        total_T += (2/3) * function(a + k*h)

    total_I = h * (total_S + 2*total_T)

    if recursing == True:
        simp_error = np.abs(total_I - AdaptiveSimpSum(function, a, b, N_steps=2*N_steps, recursing=False)) / 15
        print(f"{N_steps} steps, error: {simp_error}")
        
        if simp_error >= tolerance:
            total_I = AdaptiveSimpSum(function, a, b, N_steps=2*N_steps, tolerance=tolerance, recursing=True, last_total=(total_S + total_T))

    return total_I

print(AdaptiveSimpSum(f_2, 0, 1, tolerance=1e-6))

10 steps, error: 0.003796024366896744
20 steps, error: 0.0004731630961837263
40 steps, error: 3.44740157342686e-05
80 steps, error: 2.236766804456873e-06
160 steps, error: 1.4110396072316195e-07
0.45583027431256834
